In [ ]:
# Optional pip3 install 
#!pip3 install ipywidgets

In [ ]:
# Run this either in your Notebook or your shell.
# If you have done so already, skip it.
#!bash ./download_gutenberg.sh

In [ ]:
import time
import ray
import os

In [ ]:
ray.init(address="ray://<private_IPv4_addr_vm1>:10001")

In [ ]:
print('''This cluster consists of
    {} nodes in total
    {} CPU resources in total
    {} Memory resources in total
    {} Object store memory resources in total
    '''.format(len(ray.nodes()), ray.cluster_resources()['CPU'], 
           ray.cluster_resources()['memory'] / (1024*1024*1024), 
           ray.cluster_resources()['object_store_memory'] / (1024*1024*1024)))

In [ ]:
%%time

# You should have a mr_txt folder created after running download_gutenberg.sh. 
# Go ahead read the files into a list.
corpus = []
for file_name in os.listdir('mr_txt'):
    file_path = os.path.join('mr_txt', file_name)
    f_txt = open(file_path,'r')
    content = f_txt.read()
    filtered_content = ''.join([char if char.isalpha() or char.isspace() or char == "'" else ' ' for char in content])
    corpus.extend(filtered_content.split())

f_txt.close()

In [ ]:
# You should specify the number of map tasks and the number of reduce tasks.
# The input data is partitioned into [num_mappers] buckets.
num_map_tasks = 8
num_reduce_tasks = 2

chunk = len(corpus) // num_map_tasks
input_buckets = [
    corpus[i * chunk: (i + 1) * chunk] for i in range(num_map_tasks)
]

In [ ]:
# Task 1: Implement the map task with ray.remote()
@ray.remote(scheduling_strategy="SPREAD")
def do_map_task(map_input, num_reduce_tasks = 4):
    """
    Launches a map task on the given 'map_input' and divide the keys into buckets for 
        'num_reduce_tasks' reduce tasks, where 'num_reduce_tasks' is the number of reduce tasks.
    Steps:
    1. Initialize an empty list named intermediate_results with a length equal to num_reduce_tasks.
    2. Call the map_function on the input_bucket to emit a list of key-value (KV) pairs.
    3. Calculate which reduce task a KV pair should go to and append the KV pair to the corresponding
        intermediate results buckets. To do so:
        Iterate through each entry in the KV pair list and hash partition (with hash(key)) 
        all KV pairs across 'num_reduce_tasks' reduce tasks using the modulo (%) operation. 
    4. Return a list of intermediate result buckets. 

    Parameters:
    - map_input (list): The input data to be mapped.
    - num_reducers (int): The number of buckets to divide the 'map_input' into (default is 4).

    Returns:
    - intermediate_results (list): A list of intermediate results buckets.
    """

    intermediate_results = [list() for _ in range(num_reduce_tasks)]
    #TODO: start your implementation below this line
    kv_pairs = map_function(map_input)

    for entry in kv_pairs:
        reducer_idx = hash(entry[0]) % num_reduce_tasks
        intermediate_results[reducer_idx].append(entry)
        
    return intermediate_results

In [ ]:
# Task 2: Implement the map function
def map_function(map_input):
    """
    Converts a 'map_input' into a list of key-value pairs, where the key is the word and the 
        value is set to 1.
    Steps:
        1. Create an empty list named kvs to store key-value pairs.
        2. Iterate through each word in the input and create a (word, 1) tuple and append it to the kvs list.
        3. After processing all words, return the kvs list as the result.

    Parameters:
    - map_input (list): List of words representing a partition of the entire document txts.

    Returns:
    - list: A list of key-value pairs generated from the input document.
    """

    kv_list = list()
    #for item in map_input:
    #    print(item)
    #TODO: start your implementation below this line
    for word in map_input:
        kv_list.append((word.lower(), 1))

    return kv_list

In [ ]:
# Task 3: Implement the reduce task with ray.remote()
@ray.remote(scheduling_strategy="SPREAD")
def do_reduce_task(*my_intermediate_results):
    """
    Launches a reduce task on a list of 'my_intermediate_results' buckets assigned to this reducer.
    Steps:
        1. Iterate over each bucket in the input argument and append the KV pair to a hash table (a dict). To do so:
            Iterate each KV pair in the bucket, and append the value ('1') to its corresponding key (a word) in 
            the hash table.
        2. Iterate each item in the hash table and calculate the word count (i.e., number of '1's) for each key by calling
            reduce_function() that you will implement for Task 4.

    Parameters:
    - *my_intermediate_results: A list of 'num_map_tasks' intermediate results buckets containing (word, 1) 
        key-value pairs, where 'num_map_tasks' is the number of map tasks. 

    Returns:
    - partial_output (dict): A dict that holds the result of this reduce task, using the word as the key and its count 
        as the value.
    """

    partial_output = dict()
    #TODO: start your implementation below this line
    reduce_results = dict()
    
    for bucket in my_intermediate_results:
        for key,value in bucket:
            if key not in reduce_results:
                vals = list()
                vals.append(value)
                reduce_results[key] =  vals
            else:
                reduce_results[key].append(value)

    for key, val in reduce_results.items():
        partial_output[key] = reduce_function(val)
                
    return partial_output

In [ ]:
# Task 4: Implement the reduce function
def reduce_function(val):
    """
    Simply return the number of items in val

    Parameters:
    - val: A list of '1's.

    Returns:
    - count: number of '1's in val

    """

    count = 0
    # TODO: Start your implementation below this line
    count = len(val)
    return count

In [ ]:
"""
Driver code to launch the entire MapReduce WordCount pipeline.
While you are running this cell, keep an eye on the Ray dashboard to keep track 
    of the task execution progress.
"""

# Driver code to launch remote map tasks on the Ray cluster.
# Do not modify this cell! 
intermediate_results = [
    do_map_task.options(num_returns = num_reduce_tasks) # each do_map_task should return a list of results
    .remote(input_bucket, num_reduce_tasks) # each remote task takes in two arguments: input_bucket and num_reducers
    for input_bucket in input_buckets
]

# Driver code to launch remote reduce tasks on the Ray cluster.
# Do not modify this cell!
merged_partial_results = []
for reducer_idx in range(num_reduce_tasks):
    merged_partial_results.append(
        do_reduce_task.remote(*[map_output[reducer_idx] for map_output in intermediate_results])
    )

In [ ]:
#intermediate_results

In [ ]:
#merged_partial_results

In [ ]:
# Collect and aggregate reduce tasks' output results at the client.
wc = dict()
wc_sorted = []
for res in ray.get(merged_partial_results):
    for key, val in res.items():
        if key not in wc:
            wc[key] = val
        else:
            wc[key] += val

# Sort all word counts
wc_sorted = sorted(wc.items(), key=lambda item: item[1], reverse=True)

# Print the top 20 words (for debugging purpose)
for word, count in wc_sorted[:20]:
    print(f"{word}, {count}")

# Save wc_sorted to a file
with open('mr_wc.output', 'w') as file:
    for word, count in wc_sorted:
        file.write(f"{word} {count}\n")

In [ ]:
# Run the wc test (either a PASS or a FAIL)
!bash ./test_mr.sh